In [2]:
import matplotlib.pyplot as plt
from scipy.ndimage import label
import numpy as np

neighbors = [(0,1), (0,-1), (1,0), (-1,0)]
fence_neighbors = {
    (0,1): [(-1, 0), (1, 0)],
    (0,-1): [(-1, 0), (1, 0)],
    (1,0): [(0, -1), (0, 1)],
    (-1,0): [(0, -1), (0, 1)]
}

def determine_cost(garden, is_part_2=False): 
    plants = np.unique(garden)

    total_cost = 0
    for plant in plants: 
        # Split it into the areas which are not connected to each other
        labeled, ncomponents = label(garden == plant)
        
        for i in range(1,ncomponents+1): # Go over each component
            area = np.sum(labeled == i) # Area is simply the number of pixels in the component

            surface_area_of_fence = 0 
            placed_fences = []
            y_positions, x_positions = np.where(labeled == i) 
            
            # For each pixel, determine whether it needs a fence
            for point in zip(y_positions, x_positions):
                # Check each neighbouring pixel
                for dir_index, (dy, dx) in enumerate(neighbors):
                    ny, nx = point[0]+dy, point[1]+dx

                    # If we get out of bounds or the neighbouring pixel is not part of the component we need a fence. 
                    if ny < 0 or ny >= labeled.shape[0] or nx < 0 or nx >= labeled.shape[1] or labeled[ny, nx] != i:
                        if not is_part_2:
                            surface_area_of_fence += 1
                        else: 
                            # For part 2 we check the neighbours of the fence to see if it is connected to another fence
                            has_fence_neighbor = False
                            fence = (point[0], point[1], dir_index)

                            placed_fences.append(fence)

                            for neighbor_dy, neighbor_dx in fence_neighbors[(dy, dx)]:
                                xy, xx = point[0]+neighbor_dy, point[1]+neighbor_dx
                                if 0 <= xy < labeled.shape[0] and 0 <= xx < labeled.shape[1]:     
                                    if (xy, xx, dir_index) in placed_fences:
                                        has_fence_neighbor = True
                                        break

                            if not has_fence_neighbor:
                                surface_area_of_fence += 1
            
            total_cost += surface_area_of_fence * area

    return total_cost

garden = open('inputs/day12.txt').read().splitlines()
garden = np.array([[plant for plant in line] for line in garden])

print("Part 1:", determine_cost(garden))
print("Part 2:", determine_cost(garden, is_part_2=True))

Part 1: 1402544
Part 2: 862486
